In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import numpy as np
import pickle
import rowan
import json

In [47]:
# instantiate a chrome options object so you can set the size and headless preference
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(chrome_options=chrome_options)


/Users/Pengji/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [48]:
point_group_rotation_matrix_dict = {}
for num in range (1,33):
    driver.get('https://www.cryst.ehu.es/cgi-bin/cryst/programs/'
               'nph-point_genpos?num={}'.format(num))

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find_all('center')[1]
    table = table.find('tbody')

    n = len(table.find_all('tr',recursive=False)[2:])
    rotations = np.zeros((n,3,3))
    i = 0
    for row in table.find_all('tr',recursive=False)[2:]:
        table = row.find('table')
        value_list = re.findall(r'[-+0-9./]+', table.find('pre').string)
        value_list = [eval(item) for item in value_list]
        rotations[i, 0, :] = value_list[0:3]
        rotations[i, 1, :] = value_list[3:6]
        rotations[i, 2, :] = value_list[6:9]
        i += 1
    point_group_rotation_matrix_dict[num] = {'rotations':rotations}
   

In [49]:
with open('point_group_rotation_matrix_dict.pickle','wb') as f:
    pickle.dump(point_group_rotation_matrix_dict, f)

In [50]:
point_group_list = ['1', '-1', '2', 'm', '2/m', '222', 'mm2', 'mmm',
                    '4', '-4', '4/m', '422', '4mm', '-42m', '4/mmm', '3',
                    '-3', '32', '3m', '-3m', '6', '-6', '6/m', '622',
                    '6mm', '-6m2', '6/mmm', '23', 'm-3', '432', '-43m', 'm-3m']
num = range(1, 33)
point_group_name_dict = dict(zip(num, point_group_list))

with open('point_group_name_mapping.json', 'w') as f:
    json.dump(point_group_name_dict, f)

In [51]:
point_group_quat_dict = {}
for key, item in point_group_rotation_matrix_dict.items():
    quats = []
    n = item['rotations'].shape[0]
    for i in range (0, n):
        qtemp = rowan.from_matrix(item['rotations'][i,:,:], require_orthogonal=False)
        quats.append(qtemp.tolist())


    point_group_quat_dict[key] = quats

with open('point_group_quat_dict.json', 'w') as f:
    json.dump(point_group_quat_dict, f)  

In [53]:
# test out stored data
with open('point_group_rotation_matrix_dict.pickle', 'rb') as f:
    point_group_rotation_matrix_dict = pickle.load(f)

import json
with open('point_group_quat_dict.json', 'rb') as f:
    point_group_quat_dict = json.load(f)

In [58]:
print(point_group_name_dict[29])
m3 = point_group_quat_dict['29']
len(m3)

m-3


24

In [86]:
mbarThreem = [
    (1, [1, 1, 1]),  # 1
    (2, [0, 0, 1]),
    (2, [0, 1, 0]),
    (2, [1, 0, 0]),
    (3, [1, 1, 1]),  # 5
    (3, [-1, 1, -1]),
    (3, [1, -1, -1]),
    (3, [-1, -1, 1]),
    (-3, [1, 1, 1]),  # 9
    (-3, [1, -1, -1]),
    (-3, [-1, -1, 1]),
    (-3, [-1, 1, -1]),
    (2, [1, 1, 0]),  # 13
    (2, [1, -1, 0]),
    (-4, [0, 0, 1]),
    (4, [0, 0, 1]),
    (-4, [1, 0, 0]),  # 17
    (2, [0, 1, 1]),
    (2, [0, 1, -1]),
    (4, [1, 0, 0]),
    (4, [0, 1, 0]),  # 21
    (2, [1, 0, 1]),
    (-4, [0, 1, 0]),
    (2, [-1, 0, 1]),
]

def gen_sym_quats(operations):
    quats = []
    for operation in operations:
        qtemp = rowan.from_axis_angle(axes=operation[1],
                                      angles=2 * np.pi / operation[0])
        quats.append(qtemp.tolist())

    return quats

[[-1.0, 7.07050159149938e-17, 7.07050159149938e-17, 7.07050159149938e-17],
 [6.123233995736766e-17, 0.0, 0.0, 1.0],
 [6.123233995736766e-17, 0.0, 1.0, 0.0],
 [6.123233995736766e-17, 1.0, 0.0, 0.0],
 [0.5000000000000001, 0.5, 0.5, 0.5],
 [0.5000000000000001, -0.5, 0.5, -0.5],
 [0.5000000000000001, 0.5, -0.5, -0.5],
 [0.5000000000000001, -0.5, -0.5, 0.5],
 [0.5000000000000001, -0.5, -0.5, -0.5],
 [0.5000000000000001, -0.5, 0.5, 0.5],
 [0.5000000000000001, 0.5, 0.5, -0.5],
 [0.5000000000000001, 0.5, -0.5, 0.5],
 [6.123233995736766e-17, 0.7071067811865475, 0.7071067811865475, 0.0],
 [6.123233995736766e-17, 0.7071067811865475, -0.7071067811865475, 0.0],
 [0.7071067811865476, -0.0, -0.0, -0.7071067811865475],
 [0.7071067811865476, 0.0, 0.0, 0.7071067811865475],
 [0.7071067811865476, -0.7071067811865475, -0.0, -0.0],
 [6.123233995736766e-17, 0.0, 0.7071067811865475, 0.7071067811865475],
 [6.123233995736766e-17, 0.0, 0.7071067811865475, -0.7071067811865475],
 [0.7071067811865476, 0.70710678118

In [134]:
n = len(point_group_quat_dict['m-3'])
for i in range(0, n):
    print(rowan.allclose(point_group_quat_dict['432'][i], gen_sym_quats(mbarThreem)[i]))
    print(rowan.allclose(rowan.multiply([-1, 0, 0, 0],point_group_quat_dict['m-3'][i]), gen_sym_quats(mbarThreem)[i]))

False
True
False
True
False
True
False
True
True
False
True
False
False
True
False
True
False
True
True
False
True
False
False
True
False
False
True
False
True
False
True
False
False
False
False
False
True
False
True
False
True
False
True
False
False
False
False
False


In [132]:
gen_sym_quats(mbarThreem)

[[-1.0, 7.07050159149938e-17, 7.07050159149938e-17, 7.07050159149938e-17],
 [6.123233995736766e-17, 0.0, 0.0, 1.0],
 [6.123233995736766e-17, 0.0, 1.0, 0.0],
 [6.123233995736766e-17, 1.0, 0.0, 0.0],
 [0.5000000000000001, 0.5, 0.5, 0.5],
 [0.5000000000000001, -0.5, 0.5, -0.5],
 [0.5000000000000001, 0.5, -0.5, -0.5],
 [0.5000000000000001, -0.5, -0.5, 0.5],
 [0.5000000000000001, -0.5, -0.5, -0.5],
 [0.5000000000000001, -0.5, 0.5, 0.5],
 [0.5000000000000001, 0.5, 0.5, -0.5],
 [0.5000000000000001, 0.5, -0.5, 0.5],
 [6.123233995736766e-17, 0.7071067811865475, 0.7071067811865475, 0.0],
 [6.123233995736766e-17, 0.7071067811865475, -0.7071067811865475, 0.0],
 [0.7071067811865476, -0.0, -0.0, -0.7071067811865475],
 [0.7071067811865476, 0.0, 0.0, 0.7071067811865475],
 [0.7071067811865476, -0.7071067811865475, -0.0, -0.0],
 [6.123233995736766e-17, 0.0, 0.7071067811865475, 0.7071067811865475],
 [6.123233995736766e-17, 0.0, 0.7071067811865475, -0.7071067811865475],
 [0.7071067811865476, 0.70710678118

In [133]:
point_group_quat_dict['m-3']

[[1.0, -0.0, -0.0, -0.0],
 [0.0, -0.0, -0.0, -1.0],
 [0.0, -0.0, -1.0, -0.0],
 [0.0, -1.0, -0.0, -0.0],
 [0.5000000000000001, 0.5, 0.5000000000000001, 0.5000000000000001],
 [0.5000000000000001, -0.5, 0.5000000000000001, -0.5000000000000001],
 [-0.5000000000000001, -0.5, 0.5000000000000001, 0.5000000000000001],
 [-0.5000000000000001, 0.5, 0.5000000000000001, -0.5000000000000001],
 [-0.5000000000000001, 0.5, 0.5000000000000001, 0.5000000000000001],
 [0.5000000000000001, -0.5, 0.5000000000000001, 0.5000000000000001],
 [0.5000000000000001, 0.5, 0.5000000000000001, -0.5000000000000001],
 [-0.5000000000000001, -0.5, 0.5000000000000001, -0.5000000000000001],
 [0.0, -1.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0],
 [1.0, -0.0, -0.0, -0.0],
 [-0.28867513459481287,
  0.8660254037844387,
  -0.28867513459481287,
  -0.28867513459481287],
 [0.28867513459481287,
  0.8660254037844387,
  0.28867513459481287,
  -0.28867513459481287],
 [-0.28867513459481287,
  0.8660254037844387,
  

In [61]:
q1 = [-0.5000000000000001, 0.5, 0.5000000000000001, -0.5000000000000001]
q2 = rowan.multiply([-1, 0, 0, 0], q1)
v = [1,3,4]
print(rowan.rotate(q1, v))
print(rowan.rotate(q2, v))

[-4.  1. -3.]
[-4.  1. -3.]
